# **Unpacking original images**

In [0]:
!unzip /content/drive/My\ Drive/Thesis/images.zip

# **Image pre-processing for detecting infected cells**

Importing libraries

In [0]:
import os
from keras.preprocessing import image
from PIL import Image
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
import sys

Pre-processing

In [0]:
inputpath = '/content/images'
outputpath = '/content/drive/My Drive/Thesis/Faster_RCNN/Processed_Images_OB'
width = 1600
height = 1200

for filename in os.listdir(inputpath):
    img = Image.open(inputpath+'//'+filename).resize((width, height),Image.ANTIALIAS)
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.GaussianBlur(img, (5, 5), 10)
    out = os.path.join(outputpath, filename)
    cv2.imwrite(out, img)

# **Moving Train and Test images to respective folders**
As all original images (both train and test) were present in a single folder, to implemant Faster RCNN they were required to be moved to their respective folders.

Moving test images to test folder

In [0]:
test = pd.read_csv('/content/drive/My Drive/Thesis/CNN/Test_Cat.csv')

outputpath = "/content/drive/My Drive/Thesis/Faster_RCNN/Test"

for i in tqdm(range(test.shape[0])):
    img = Image.open('/content/drive/My Drive/Thesis/Faster_RCNN/Processed_Images_OB/'+str(test['image'][i])+'.png')
    img = image.img_to_array(img)
    out = os.path.join(outputpath, str(test['image'][i])+'.png')
    cv2.imwrite(out, img)

100%|██████████| 120/120 [00:08<00:00, 15.01it/s]


Moving train images to train folder

In [0]:
train = pd.read_csv('/content/drive/My Drive/Thesis/CNN/Train_Cat.csv')

outputpath = "/content/drive/My Drive/Thesis/Faster_RCNN/Train"

for i in tqdm(range(train.shape[0])):
    img = Image.open('/content/drive/My Drive/Thesis/Faster_RCNN/Processed_Images_OB/'+str(train['image'][i])+'.png')
    img = image.img_to_array(img)
    out = os.path.join(outputpath, str(train['image'][i])+'.png')
    cv2.imwrite(out, img)

100%|██████████| 1208/1208 [01:43<00:00, 11.68it/s]


# **Create Annotate.txt**
It was one of the requirements to create a text file in the format (filepath, xmin, ymin, xmax, ymax, class) which will later be used for training the model. Here OB_Training csv file contains bounding box co-ordinates only for infected cells because the model willl be trained to find only infected cells.

In [0]:
train = pd.read_csv('/content/drive/My Drive/Thesis/Faster_RCNN/OB_Training.csv')

data = pd.DataFrame()
data['format'] = train['image']

# as the images are in train_images folder, add train_images before the image name
for i in range(data.shape[0]):
    data['format'][i] = '/content/drive/My Drive/Thesis/Faster_RCNN/Train//' + data['format'][i] + '.png'

# add xmin, ymin, xmax, ymax and class as per the format required
for i in range(data.shape[0]):
    data['format'][i] = data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['category'][i]

data.to_csv('/content/drive/My Drive/Thesis/Faster_RCNN/annotate.txt', header=None, index=None, sep=' ')

with open('/content/drive/My Drive/Thesis/Faster_RCNN/annotate.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"', '')

# Write the file out again
with open('/content/drive/My Drive/Thesis/Faster_RCNN/annotate.txt', 'w') as file:
  file.write(filedata)

# **Downgrading Keras version and changing current working directory**
While training, model throws an error for latest keras version. So there was a need to downgrade keras to version 2.2.0

In [0]:
!python -c 'import keras; print(keras.__version__)'
!pip install keras==2.2.0
!python -c 'import keras; print(keras.__version__)'

Changing current working directory to Faster_RCNN because train.py, test.py and other python files used for executing model are present there

In [0]:
#!rm -rf /content/keras-frcnn-master-infected

%cd /content/drive/My\ Drive/Thesis/Faster_RCNN

!pwd

/content/drive/My Drive/Thesis/Faster_RCNN
/content/drive/My Drive/Thesis/Faster_RCNN


# **Scripts for training and testing the model**

Script to train the model using annotate.txt

In [0]:
!python train_frcnn.py -o simple -p annotate.txt

Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0,
 'difficult': 441,
 'gametocyte': 144,
 'ring': 353,
 'schizont': 179,
 'trophozoite': 1473}
Num classes (including bg) = 6
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 844
Num val samples 175
W0812 00:51:51.024202 140707235104640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 00:51:51.033706 140707235104640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 00:51:51.038258 140707235104640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4076: The name tf.

Script to test the model on test images

In [0]:
!python test_frcnn.py -p Test

Using TensorFlow backend.
{0: 'difficult', 1: 'schizont', 2: 'ring', 3: 'trophozoite', 4: 'gametocyte', 5: 'bg'}
W0812 01:52:08.893526 139803151189888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 01:52:08.904017 139803151189888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 01:52:08.908555 139803151189888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 01:52:08.938715 139803151189888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3900: The name tf.nn.max_